In [1]:
import glob
import zipfile
import pandas as pd
import gc

In [8]:
columns = ['LP1', 'Data', 'Czas', 'LP2','Wartosc','Jakosc_pomiaru']
df=pd.DataFrame(columns=columns)
print(df)

#pętla wczytująca wszystkie csv z zipów z jednego katalogu

for zip_file in glob.glob("C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05/*.zip"):
    zf = zipfile.ZipFile(zip_file)
    dfs = [pd.read_csv(zf.open(f), sep='|', skiprows=4, names=columns) for f in zf.namelist()]
    df = df.append(dfs)
#    df = pd.concat(dfs,ignore_index=False)
    print(zip_file)
    
# resetowanie indeksu
df=df.reset_index()

# Wyodrębnianie informacji z pierwszej kolumny
df['param']=df['index'].str.slice(start=17, stop=28)
df['Analog/Digital']=df['index'].str.slice(start=28, stop=29)
df['jednostka']=df['index'].str.slice(start=29, stop=32)
df['param+jednostka']=df['param'] + df['jednostka']

#przekształcenie daty i czasu na typ datetime
df['Data_time']=df.Data+' '+df.Czas
df['Data_time']=pd.to_datetime(df['Data_time'], format='%Y/%m/%d %H:%M:%S.%f')

#usunięcie zbędnych kolumn
df.drop(labels=['param','jednostka','Data','Czas'], axis=1, inplace=True)

print('Skończono transformację danych')
#odfiltrowanie prawidłowych pomiarów
df=df[df.Jakosc_pomiaru == 192]

df_pivot=pd.pivot_table(df, index='Data_time',columns='param+jednostka', values='Wartosc', aggfunc='mean')

print('skończono pivot')

df_pivot.to_csv('C:/Users/E7240/OneDrive/cuvalley/pivot2020_05.csv')
print('csv ready')

Empty DataFrame
Columns: [LP1, Data, Czas, LP2, Wartosc, Jakosc_pomiaru]
Index: []
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200501.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200502.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200503.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200504.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200505.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200506.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200507.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200508.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 175L/2020/05\KLDSMG_WOS___175L_20200509.zip
C:/Users/E7240/Documents/cuvalley/MonitoringSMG/WOS 1

In [7]:
# kasowanie zmiennych i czyszczenie pamięci

del df_pivot
del df
gc.collect()

58